In [1]:
import pandas as pd
import numpy as np

### Для объединения всех подготовленных файлов в 1 для упращения всех дальнейщей работ 

In [2]:
pd.set_option('display.max_rows', 1750)

In [3]:
df_16_17 = pd.read_excel("data_preparation/files_client/16-17.xlsx")
df_17_18 = pd.read_excel("data_preparation/files_client/17-18.xlsx")
df_18_19 = pd.read_excel("data_preparation/files_client/18-19.xlsx")
df_19_20 = pd.read_excel("data_preparation/files_client/19-20.xlsx")
df_20_21 = pd.read_excel("data_preparation/files_client/20-21.xlsx")
df_21_22 = pd.read_excel("data_preparation/files_client/21-22.xlsx")
df_22_23 = pd.read_excel("data_preparation/files_client/22-23.xlsx")
df_23_24 = pd.read_excel("data_preparation/files_client/23-24.xlsx")

#### Изменения типа данных у колонки

In [4]:
df_list = [df_16_17,df_17_18,df_18_19,df_19_20,df_20_21,df_21_22,df_22_23,df_23_24]
for i in range(0,len(df_list)):
    df_list[i] = df_list[i].dropna(subset='phone_1')
    df_list[i]['phone_1'] = df_list[i]['phone_1'].astype(int)
df_16_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name_user  279 non-null    object 
 1   phone      281 non-null    object 
 2   city       281 non-null    object 
 3   addres     272 non-null    object 
 4   kids       268 non-null    object 
 5   status     262 non-null    object 
 6   source     46 non-null     object 
 7   call       276 non-null    object 
 8   note       99 non-null     object 
 9   phone_1    281 non-null    int64  
 10  phone_2    46 non-null     float64
 11  phone_3    4 non-null      float64
 12  name_1     275 non-null    object 
 13  name_2     16 non-null     object 
 14  name_3     8 non-null      object 
 15  ages       232 non-null    object 
 16  avg_age    232 non-null    float64
dtypes: float64(3), int64(1), object(13)
memory usage: 37.4+ KB


/var/folders/38/l4hfq4694136377jqp7rr75c0000gn/T/ipykernel_57728/3658572446.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list[i]['phone_1'] = df_list[i]['phone_1'].astype(int)


### Объединение файлов без колонок изменяющие каждый год - звонок клиенту, возраст. Что бы захватить всех клиентов, которые по каким либо причинам были удалены

In [5]:
df = pd.DataFrame(columns=['name_user','phone','name_1','phone_1','name_2','phone_2',
                           'name_3','phone_3','city','addres','kids','status','source'])
df_list = [df_16_17,df_17_18,df_18_19,df_19_20,df_20_21,df_21_22,df_22_23,df_23_24]
for df_i in df_list:
    df['phone_1'] = df['phone_1'].astype(int)
    df_i = df_i.drop(['call','note','ages','avg_age'],axis=1)
    df_i = df_i.dropna(subset='phone_1')
    df_i['phone_1'] = df_i['phone_1'].astype(int)
    df = pd.concat([df,df_i], axis=0, join='outer')
#df.info()

#### Сохранение промежуточного файла

In [6]:
df.to_excel('concat_client.xlsx', index=False)

In [7]:
df_concat = pd.read_excel('concat_client.xlsx')

#### Удаление дублей

In [8]:
# df_concat.loc[df_concat.duplicated(subset='phone_1',keep='last') == True]

In [9]:
len(df_concat)

3424

In [10]:
len(df_concat) - len(df_concat.drop_duplicates())

2115

In [11]:
df_concat.drop_duplicates(subset='phone_1',keep='last',inplace=True)

In [12]:
len(df_concat)

687

In [13]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687 entries, 333 to 3423
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name_user  685 non-null    object 
 1   phone      687 non-null    object 
 2   name_1     678 non-null    object 
 3   phone_1    687 non-null    int64  
 4   name_2     24 non-null     object 
 5   phone_2    88 non-null     float64
 6   name_3     9 non-null      object 
 7   phone_3    7 non-null      float64
 8   city       687 non-null    object 
 9   addres     677 non-null    object 
 10  kids       649 non-null    object 
 11  status     680 non-null    object 
 12  source     247 non-null    object 
dtypes: float64(2), int64(1), object(10)
memory usage: 75.1+ KB


#### Сохранение промежуточного файла

In [14]:
df_concat.to_excel('drop_dupli_client.xlsx', index=False)

In [15]:
df_dubl = pd.read_excel('drop_dupli_client.xlsx')

### Добавление данных по годам для полного списка клиентов

In [16]:
df_list = [df_16_17,df_17_18,df_18_19,df_19_20,df_20_21,df_21_22,df_22_23,df_23_24]
name_df = ['_16_17','_17_18','_18_19','_19_20','_20_21','_21_22','_22_23','_23_24']
df_fin = df_dubl.copy()
for i in range(0,len(df_list)):
    df_i = df_list[i].drop(['name_user','phone','name_1','name_2','phone_2',
                           'name_3','phone_3','city','addres','kids','status','source','note'],axis=1)
#     df_i.to_excel('intermediate/'+name_df[i]+'.xlsx', index=False)
    df_i.drop_duplicates(subset='phone_1',keep='last',inplace=True)
    df_fin = df_fin.merge(df_i, left_on='phone_1', right_on='phone_1', 
                           how='left', suffixes=('', name_df[i]))
#     df_fin.info()
df_fin.rename(columns = {'call': 'call_16_17', 'ages': 'ages_16_17', 'avg_age': 'avg_age_16_17'},inplace=True)
# df_fin.info()

In [17]:
df_fin.to_excel('full_client.xlsx', index=False)

#### Проверка на дубли

In [18]:
df_fin.loc[df_fin.duplicated() == True]

,name_user,phone,name_1,phone_1,name_2,phone_2,name_3,phone_3,city,addres,...,avg_age_20_21,call_21_22,ages_21_22,avg_age_21_22,call_22_23,ages_22_23,avg_age_22_23,call_23_24,ages_23_24,avg_age_23_24


In [19]:
df = df_fin

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687 entries, 0 to 686
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name_user      685 non-null    object 
 1   phone          687 non-null    object 
 2   name_1         678 non-null    object 
 3   phone_1        687 non-null    int64  
 4   name_2         24 non-null     object 
 5   phone_2        88 non-null     float64
 6   name_3         9 non-null      object 
 7   phone_3        7 non-null      float64
 8   city           687 non-null    object 
 9   addres         677 non-null    object 
 10  kids           649 non-null    object 
 11  status         680 non-null    object 
 12  source         247 non-null    object 
 13  call_16_17     273 non-null    object 
 14  ages_16_17     231 non-null    object 
 15  avg_age_16_17  231 non-null    float64
 16  call_17_18     350 non-null    object 
 17  ages_17_18     303 non-null    object 
 18  avg_age_17

#### Редактирование возраста с исходом первоначального значения, для заполнения некорректных и пустых значений

In [21]:
df = df_fin[['ages_16_17', 'ages_17_18', 'ages_18_19', 'ages_19_20', 'ages_20_21', 
             'ages_21_22', 'ages_22_23', 'ages_23_24']]

In [22]:
# df - просмотр с чем рабоем

In [23]:
def increasing_the_age(column_1, column_2):
    column_n=[]
    for row in df[column_1]:
        row_n=[]
        if not pd.isnull(row):
            list_r = row.strip('][').split(', ')
            for i in list_r:
                row_n.append(float(i)+1)
        else:
            row_n.append(np.nan)
        column_n.append(str(row_n)) 
    
    for i in range(0,len(df[column_2])):
        if column_n[i] != '[nan]':
                df[column_2][i] = column_n[i]
            
            
increasing_the_age('ages_16_17', 'ages_17_18')
increasing_the_age('ages_17_18', 'ages_18_19')
increasing_the_age('ages_18_19', 'ages_19_20')
increasing_the_age('ages_19_20', 'ages_20_21')
increasing_the_age('ages_20_21', 'ages_21_22')
increasing_the_age('ages_21_22', 'ages_22_23')
increasing_the_age('ages_22_23', 'ages_23_24')

/var/folders/38/l4hfq4694136377jqp7rr75c0000gn/T/ipykernel_57728/2759226717.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_2][i] = column_n[i]


In [24]:
# df - просмотр с чем рабоем

In [25]:
df_fin[['ages_16_17', 'ages_17_18', 'ages_18_19', 'ages_19_20', 'ages_20_21', 
             'ages_21_22', 'ages_22_23', 'ages_23_24']] = df

In [26]:
# df_fin - просмотр с чем рабоем

#### Аналогично что делали с возрастом

In [27]:
df = df_fin[['avg_age_16_17', 'avg_age_17_18', 'avg_age_18_19', 'avg_age_19_20', 'avg_age_20_21', 
             'avg_age_21_22', 'avg_age_22_23', 'avg_age_23_24']]

In [28]:
# df - просмотр с чем рабоем

In [29]:
def increasing_the_age_avg(column_1, column_2):
    column_n=[]
    for row in df[column_1]:
#         row_n=[]
        if not pd.isnull(row):
            column_n.append(float(row)+1)
        else:
            column_n.append(np.nan)
#         column_n.append(row_n) 
#     print(row_n)
    
    for i in range(0,len(df[column_2])):
        if not pd.isnull(column_n[i]):
                df[column_2][i] = column_n[i]
            
            
increasing_the_age_avg('avg_age_16_17', 'avg_age_17_18')
increasing_the_age_avg('avg_age_17_18', 'avg_age_18_19')
increasing_the_age_avg('avg_age_18_19', 'avg_age_19_20')
increasing_the_age_avg('avg_age_19_20', 'avg_age_20_21')
increasing_the_age_avg('avg_age_20_21', 'avg_age_21_22')
increasing_the_age_avg('avg_age_21_22', 'avg_age_22_23')
increasing_the_age_avg('avg_age_22_23', 'avg_age_23_24')

/var/folders/38/l4hfq4694136377jqp7rr75c0000gn/T/ipykernel_57728/3654623505.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_2][i] = column_n[i]


In [30]:
# df - просмотр с чем рабоем

In [31]:
df_fin[['avg_age_16_17', 'avg_age_17_18', 'avg_age_18_19', 'avg_age_19_20', 'avg_age_20_21', 
             'avg_age_21_22', 'avg_age_22_23', 'avg_age_23_24']] = df

In [32]:
# df_fin - просмотр с чем рабоем

In [33]:
df_fin.to_excel('full_client.xlsx', index=False)

In [34]:
df = df_fin[['call_23_24', 'call_22_23', 'call_21_22', 'call_20_21', 
             'call_19_20', 'call_18_19', 'call_17_18', 'call_16_17' ]]

In [35]:
# df - просмотр с чем рабоем

#### Создание колонки с количеством заказов

In [36]:
new_row = [0] * 687
for column in df:
    for i in range(0,len(df[column])):
        if df[column][i] in ['Заказал', 'Отменил']:
            new_row[i] = new_row[i] + 1
df_fin['number_orders'] = new_row

In [37]:
df_fin['number_orders'].value_counts()

1    367
2    120
0     74
3     66
4     32
5     14
7      7
6      7
Name: number_orders, dtype: int64

In [38]:
df_fin['id'] = df_fin.index

In [39]:
# df_fin - просмотр с чем рабоем

In [40]:
df_fin.to_excel('full_client.xlsx', index=False)